<a href="https://colab.research.google.com/github/wnsports/SportsAnalyticsNotebook/blob/main/tutorials/socceraction_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 内容

- socceractionのインストール
- statsbombのデータを利用したActionデータへの置き換え
- Actionデータの可視化
- Action Valueを用いた行動評価値モデルのトレーニングとそのモデルを用いた解析

Installation

In [ ]:
!pip install -q socceraction
!pip install -q statsbombpy

In [ ]:
!pip install -q matplotsoccer

Load Event Data from statsbomb or wyscout

In [1]:
import pandas as pd
from socceraction.data.statsbomb import StatsBombLoader

# Set up the StatsBomb data loader
SBL = StatsBombLoader()

# View all available competitions
df_competitions = SBL.competitions()

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [5]:
set(df_competitions["competition_id"].values)

{2, 7, 9, 11, 12, 16, 35, 37, 43, 44, 49, 53, 55, 72, 81, 87, 116, 1238, 1470}

In [ ]:
# Create a dataframe with all games from the 2018 World Cup
df_games = SBL.games(competition_id=43, season_id=3).set_index("game_id")
df_games.head()

Converting SPADL data

In [ ]:
import socceraction.spadl as spadl

# df_teams, df_players, df_events, df_gamesあたりのデータセットの作成の仕方について学ぶ

game_id = 8657
df_teams = SBL.teams(game_id)
df_players = SBL.players(game_id)
df_events = SBL.events(game_id)

home_team_id = df_games.at[game_id, "home_team_id"]
df_actions = spadl.statsbomb.convert_to_actions(df_events, home_team_id)

In [ ]:
# start_x, start_y, end_x, end_y, type_id, result_id, bodypart_id, action_id
df_actions.head()

In [ ]:
# visualize soccer
import matplotsoccer as mps

# Select relevant actions
df_actions_goal = df_actions.loc[200:225]
# Replace result, actiontype and bodypart IDs by their corresponding name
df_actions_goal = spadl.add_names(df_actions_goal)
# Add team and player names
df_actions_goal = df_actions_goal.merge(df_teams).merge(df_players)
# Create the plot
mps.actions(
    location=df_actions_goal[["start_x", "start_y", "end_x", "end_y"]],
    action_type=df_actions_goal.type_name,
    team=df_actions_goal.team_name,
    result=df_actions_goal.result_name == "success",
    label=df_actions_goal[["time_seconds", "type_name", "player_name", "team_name"]],
    labeltitle=["time", "actiontype", "player", "team"],
    zoom=False
)

Valuing Actions

In [ ]:
# その行動の寄与度について3つの評価のフレームワークを準備 -> xT, VAEP, Atomic-Vaep
# This time, xT
# xTは、シーズンとかリーグを通して学習されるもの
# 今回のxTについてはあるリーグの通年のものから算出されたM*N Gridを用いる

In [13]:
import socceraction.xthreat as xthreat

url_grid = "https://karun.in/blog/data/open_xt_12x8_v1.json"
xT_model = xthreat.load_model(url_grid)

In [15]:
df_actions_ltr = spadl.play_left_to_right(df_actions, home_team_id)
df_actions["xT_value"] = xT_model.rate(df_actions_ltr)

In [ ]:
# Create the plot
# Select relevant actions
df_actions_goal = df_actions.loc[100:125]
# Replace result, actiontype and bodypart IDs by their corresponding name
df_actions_goal = spadl.add_names(df_actions_goal)
# Add team and player names
df_actions_goal = df_actions_goal.merge(df_teams).merge(df_players)

mps.actions(
    location=df_actions_goal[["start_x", "start_y", "end_x", "end_y"]],
    action_type=df_actions_goal.type_name,
    team=df_actions_goal.team_name,
    result=df_actions_goal.result_name == "success",
    label=df_actions_goal[["time_seconds", "type_name", "player_name", "team_name", "xT_value"]],
    labeltitle=["time", "actiontype", "player", "team", "xT"],
    zoom=False
)

Training own xT model

In [ ]:
import pandas as pd
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat

# 1. Load a set of actions to train the model on
SBL = StatsBombLoader()
df_games = SBL.games(competition_id=43, season_id=3)

dataset = [
    {
        **game,
        'actions': spadl.statsbomb.convert_to_actions(
            events=SBL.events(game['game_id']),
            home_team_id=game['home_team_id']
        )
    }
    for game in df_games.to_dict(orient='records')
]

In [2]:
# 2. Convert direction of play + add names
df_actions_ltr = pd.concat([
  spadl.play_left_to_right(game['actions'], game['home_team_id'])
  for game in dataset
])
df_actions_ltr = spadl.add_names(df_actions_ltr)

In [3]:
# 3. Train xT model with 16 x 12 grid
xTModel = xthreat.ExpectedThreat(l=16, w=12)
xTModel.fit(df_actions_ltr)

# iterations:  44


In [ ]:
# Visualize xT model with soccer plot
import matplotsoccer as mps
game_id = 8657

df_teams = SBL.teams(game_id)
df_players = SBL.players(game_id)
df_events = SBL.events(game_id)

df_games_index = df_games.set_index("game_id")
home_team_id = df_games_index.at[game_id, "home_team_id"]
df_actions = spadl.statsbomb.convert_to_actions(df_events, home_team_id)
df_actions_ltr = spadl.play_left_to_right(df_actions, home_team_id)
df_actions["xT_value"] = xTModel.rate(df_actions_ltr)

# Select relevant actions
df_actions_goal = df_actions.loc[100:125]
# Replace result, actiontype and bodypart IDs by their corresponding name
df_actions_goal = spadl.add_names(df_actions_goal)
# Add team and player names
df_actions_goal = df_actions_goal.merge(df_teams).merge(df_players)

mps.actions(
    location=df_actions_goal[["start_x", "start_y", "end_x", "end_y"]],
    action_type=df_actions_goal.type_name,
    team=df_actions_goal.team_name,
    result=df_actions_goal.result_name == "success",
    label=df_actions_goal[["time_seconds", "type_name", "player_name", "team_name", "xT_value"]],
    labeltitle=["time", "actiontype", "player", "team", "xT"],
    zoom=False
)